# BEHRT MLM with new data

```This is a modified version of the MLM.ipynb notebook```

This notebook has been changed to run new benchmarking datasets


In [1]:
dataset = "local_example"

In [2]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/BEHRT-with-FastEHR/my-virtual-env-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

%load_ext autoreload
%autoreload 2

print(os.getcwd())

Added path '/rds/homes/g/gaddcz/Projects/BEHRT-with-FastEHR/my-virtual-env-icelake/lib/python3.10/site-packages' at start of search paths.
/rds/homes/g/gaddcz/Projects/BEHRT-with-FastEHR/task


In [3]:
import sys 
sys.path.insert(0, '../')

In [4]:
from common.common import create_folder
from common.pytorch import load_model
import pytorch_pretrained_bert as Bert
from model.utils import age_vocab
from common.common import load_obj
from dataLoader.MLM import MLMLoader
from torch.utils.data import DataLoader
import pandas as pd
from model.MLM import BertForMaskedLM
from model.optimiser import adam
import sklearn.metrics as skm
import numpy as np
import torch
import time
import torch.nn as nn
import os

In [5]:
class BertConfig(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfig, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings = config.get('max_position_embedding'),
            initializer_range=config.get('initializer_range'),
        )
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        
class TrainConfig(object):
    def __init__(self, config):
        self.batch_size = config.get('batch_size')
        self.use_cuda = config.get('use_cuda')
        self.max_len_seq = config.get('max_len_seq')
        self.train_loader_workers = config.get('train_loader_workers')
        self.test_loader_workers = config.get('test_loader_workers')
        self.device = config.get('device')
        self.output_dir = config.get('output_dir')
        self.output_name = config.get('output_name')
        self.best_name = config.get('best_name')

### Decide which dataset to use

In [6]:

match dataset:
    case "local_example":
        file_config = {
            'vocab':'data/local_example/token2idx',  # vocabulary idx2token, token2idx
            'data': 'data/local_example/data_train.parquet',  # formated data 
            'model_path': f'data/{dataset}/', # where to save model
            'model_name': 'local_MLM-notebook.ckpt', # model name\
            'file_name': 'local_MLM-notebook.out',  # log path
        }
    case "fastehr_example":
        file_config = {
            'vocab':'/rds/homes/g/gaddcz/Projects/FastEHR/examples/data/_built/adapted/BEHRT/T2D_hypertension/token2idx',  # vocabulary idx2token, token2idx
            'data': '/rds/homes/g/gaddcz/Projects/FastEHR/examples/data/_built/adapted/BEHRT/T2D_hypertension/dataset.parquet',  # formated data 
            'model_path': f'data/{dataset}/', # where to save model
            'model_name': 'fastehr_MLM-notebook.ckpt', # model name\
            'file_name': 'fastehr_MLM-notebook.out',  # log path
        }
    case "hypertension":
        file_config = {
            'vocab':'/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/BEHRT/token2idx',  # vocabulary idx2token, token2idx
            'data': '/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/BEHRT/train_dataset.parquet',  # formated data 
            'model_path': f'data/{dataset}/', # where to save model
            'model_name': 'hypertension_MLM-notebook.ckpt', # model name\
            'file_name': 'hypertension_MLM-notebook.out',  # log path
        }
    case "cvd":
        file_config = {
            'vocab':'/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/BEHRT/token2idx',  # vocabulary idx2token, token2idx
            'data': '/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/BEHRT/train_dataset.parquet',  # formated data 
            'model_path': f'data/{dataset}/', # where to save model
            'model_name': 'cvd_MLM-notebook.ckpt', # model name\
            'file_name': 'cvd_MLM-notebook.out',  # log path
        }
    
    case _:
        raise NotImplementedError
        
create_folder(file_config['model_path'])
print(dataset)

local_example


In [7]:
global_params = {
    'max_seq_len': 64,
    'max_age': 110,
    'month': 1,
    'age_symbol': None,
    'min_visit': 1,                        # Reduced to one (from 5) to be comparable
    'gradient_accumulation_steps': 1
}

optim_param = {
    'lr': 3e-5,
    'warmup_proportion': 0.1,
    'weight_decay': 0.01
}

train_params = {
    'batch_size': 256,
    'use_cuda': True,
    'max_len_seq': global_params['max_seq_len'],
    'device': 'cuda:0'
}

### Create dataset and dataloaders

In [8]:
BertVocab = load_obj(file_config['vocab'])
ageVocab, _ = age_vocab(max_age=global_params['max_age'], mon=global_params['month'], symbol=global_params['age_symbol'])

In [9]:
data = pd.read_parquet(file_config['data'])
# remove patients with visits less than min visit
data['length'] = data['caliber_id'].apply(lambda x: len([i for i in range(len(x)) if x[i] == 'SEP']))
data = data[data['length'] >= global_params['min_visit']]
data = data.reset_index(drop=True)

In [10]:
Dset = MLMLoader(data, BertVocab['token2idx'], ageVocab, max_len=train_params['max_len_seq'], code='caliber_id')
trainload = DataLoader(dataset=Dset, batch_size=train_params['batch_size'], shuffle=True, num_workers=3)

### Define model

In [11]:
model_config = {
    'vocab_size': len(BertVocab['token2idx'].keys()), # number of disease + symbols for word embedding
    'hidden_size': 288, # word embedding and seg embedding hidden size
    'seg_vocab_size': 2, # number of vocab for seg embedding
    'age_vocab_size': len(ageVocab.keys()), # number of vocab for age embedding
    'max_position_embedding': train_params['max_len_seq'], # maximum number of tokens
    'hidden_dropout_prob': 0.1, # dropout rate
    'num_hidden_layers': 6, # number of multi-head attention layers required
    'num_attention_heads': 12, # number of attention heads
    'attention_probs_dropout_prob': 0.1, # multi-head attention dropout rate
    'intermediate_size': 512, # the size of the "intermediate" layer in the transformer encoder
    'hidden_act': 'gelu', # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
    'initializer_range': 0.02, # parameter weight initializer range
}

In [12]:
conf = BertConfig(model_config)
model = BertForMaskedLM(conf)

In [13]:
model = model.to(train_params['device'])
optim = adam(params=list(model.named_parameters()), config=optim_param)

t_total value of -1 results in schedule not being applied


### Train model using BEHRT's experiment setup

In [14]:
def cal_acc(label, pred):
    logs = nn.LogSoftmax()
    label=label.cpu().numpy()
    ind = np.where(label!=-1)[0]
    truepred = pred.detach().cpu().numpy()
    truepred = truepred[ind]
    truelabel = label[ind]
    truepred = logs(torch.tensor(truepred))
    outs = [np.argmax(pred_x) for pred_x in truepred.numpy()]
    precision = skm.precision_score(truelabel, outs, average='micro')
    return precision

In [15]:
def train(e, loader):
    tr_loss = 0
    temp_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    cnt= 0
    start = time.time()

    for step, batch in enumerate(loader):
        cnt +=1
        batch = tuple(t.to(train_params['device']) for t in batch)
        age_ids, input_ids, posi_ids, segment_ids, attMask, masked_label = batch
        loss, pred, label = model(input_ids, age_ids, segment_ids, posi_ids,attention_mask=attMask, masked_lm_labels=masked_label)
        if global_params['gradient_accumulation_steps'] >1:
            loss = loss/global_params['gradient_accumulation_steps']
        loss.backward()
        
        temp_loss += loss.item()
        tr_loss += loss.item()
        
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        
        if step % 200==0:
            print("epoch: {}\t| cnt: {}\t|Loss: {}\t| precision: {:.4f}\t| time: {:.2f}".format(e, cnt, temp_loss/2000, cal_acc(label, pred), time.time()-start))
            temp_loss = 0
            start = time.time()
            
        if (step + 1) % global_params['gradient_accumulation_steps'] == 0:
            optim.step()
            optim.zero_grad()

    print("** ** * Saving fine - tuned model ** ** * ")
    model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
    create_folder(file_config['model_path'])
    output_model_file = os.path.join(file_config['model_path'], file_config['model_name'])

    torch.save(model_to_save.state_dict(), output_model_file)
        
    cost = time.time() - start
    return tr_loss, cost

In [16]:
data_len = 1000

f = open(os.path.join(file_config['model_path'], file_config['file_name']), "w")
f.write('{}\t{}\t{}\n'.format('epoch', 'loss', 'time'))
for e in range(50):
    loss, time_cost = train(e, trainload)
    loss = loss/data_len
    f.write('{}\t{}\t{}\n'.format(e, loss, time_cost))
f.close()    

/tmp/ipykernel_3240254/589800408.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  truepred = logs(torch.tensor(truepred))
/rds/homes/g/gaddcz/Projects/BEHRT-with-FastEHR/my-virtual-env-icelake/lib/python3.10/site-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1485.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


epoch: 0	| cnt: 1	|Loss: 0.0012827775478363038	| precision: 0.0057	| time: 1.49
** ** * Saving fine - tuned model ** ** * 


/tmp/ipykernel_3240254/589800408.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  truepred = logs(torch.tensor(truepred))


epoch: 1	| cnt: 1	|Loss: 0.000769543170928955	| precision: 0.4615	| time: 0.38
** ** * Saving fine - tuned model ** ** * 


/tmp/ipykernel_3240254/589800408.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  truepred = logs(torch.tensor(truepred))


...

epoch: 47	| cnt: 1	|Loss: 0.00056022047996521	| precision: 0.5277	| time: 0.37
** ** * Saving fine - tuned model ** ** * 


/tmp/ipykernel_3240254/589800408.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  truepred = logs(torch.tensor(truepred))


epoch: 48	| cnt: 1	|Loss: 0.0005666314959526062	| precision: 0.5316	| time: 0.35
** ** * Saving fine - tuned model ** ** * 


/tmp/ipykernel_3240254/589800408.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  truepred = logs(torch.tensor(truepred))


epoch: 49	| cnt: 1	|Loss: 0.0005858151316642761	| precision: 0.5366	| time: 0.34
** ** * Saving fine - tuned model ** ** * 
